# CSV Preprocessing

In [2]:
import pandas as pd
import numpy as np

In [50]:
country_code = [('Germany', 'de'), ('France', 'fr'), ('Belgium', 'be'), ('Denmark', 'dk'), ('Italy', 'it'), ('England', 'gb'), ('Spain', 'es'), 
                ('Portugal', 'pt'), ('Switzerland', 'ch'), ('Netherlands', 'nl')]

for country, code in country_code:
    data_set_tot = pd.DataFrame()
    for n, (i, j) in enumerate([('01', '09'), ('01', '16'), ('01', '23'), ('01', '30'), ('02', '06'), ('02', '13'), ('02', '20'), ('02', '27'), ('03', '06'), ('03', '13'), ('03', '20'), ('03', '27')]):
        data = pd.read_csv(f'{country}/regional-{code}-weekly-2025-{i}-{j}.csv')
        data["week"] = n+1
        data_set_tot = pd.concat([data_set_tot, data], axis = 0)

    data_set_tot['country'] = country
    data_set_tot.to_csv(f'Country_csv/{country}.csv')

In [49]:
data = pd.read_csv('Country_csv/England.csv')
data["country"] = 'United Kingdom'
data.to_csv('Country_csv/England.csv')

## Altair Test cells

In [5]:
import altair as alt
from vega_datasets import data

countries = alt.topo_feature(data.world_110m.url, 'countries')

alt.Chart(countries).mark_geoshape(
    fill='#666666',
    stroke='white'
).project(
    type= 'mercator',
    scale= 350,                          # Magnify
    center= [20,50],                     # [lon, lat]
    clipExtent= [[0, 0], [400, 300]],    # [[left, top], [right, bottom]]
).properties(
    title='Europe (Mercator)',
    width=400, height=300
)

alt.Chart(...)

In [20]:
import pandas as pd
import altair as alt
from vega_datasets import data

# Load the data
data_files = {
    "Germany": "Country_csv/Germany.csv",
    "France": "Country_csv/France.csv",
    "Denmark": "Country_csv/Denmark.csv",
    "Belgium": "Country_csv/Belgium.csv"
}

# Load CSVs into a dictionary of dataframes
dataframes = {country: pd.read_csv(file) for country, file in data_files.items()}

# -- Mapping of ISO 3166-1 alpha-3 codes to country names --
iso_to_country = {
    'DEU': 'Germany',
    'FRA': 'France',
    'DNK': 'Denmark',
    'BEL': 'Belgium'
}
country_to_iso = {v: k for k, v in iso_to_country.items()}

# -- Load and filter TopoJSON for just those 4 countries --
countries = alt.topo_feature(data.world_110m.url, 'countries')
selected_iso = list(iso_to_country.keys())

# -- Selection (via dropdown for compatibility with Streamlit) --
selected_country = 'Belgium'
selected_iso_code = country_to_iso[selected_country]

# -- Display Altair map (highlight selected country) --

test_df = pd.DataFrame([['France', 1]], columns=['name', 'Value'])

test_df


,name,Value
0,France,1


In [21]:

# Load the ISO-3166 contry code data
ids = pd.read_json('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/refs/heads/master/all/all.json')
ids['geo'] = ids['alpha-3'].str.lower()

test_df = test_df.merge(ids[['name', 'country-code']], how='left', on='name')

ids.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,geo
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,,142,034,,afg
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,,150,154,,ala
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,,150,039,,alb
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,,002,015,,dza
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,,009,061,,asm


In [22]:
# Merge with the gapminder2000 data frame
# Merge gapminder dataset with the latitude and longitude coordinates


background = alt.Chart(countries).mark_geoshape(fill = 'lightgrey', stroke = 'grey').project(
    type= 'mercator',
    scale= 350,                          # Magnify
    center= [20,50],                     # [lon, lat]
    clipExtent= [[0, 0], [400, 300]],    # [[left, top], [right, bottom]]
).properties(
    title='Europe (Mercator)',
    width=400, height=300
)

main = alt.Chart(countries).mark_geoshape(stroke = 'grey').transform_lookup(
    lookup = 'id', from_=alt.LookupData(data = test_df, key = 'country-code', fields = ['Value', 'Country'])
).encode(
    color = 'Value:Q',
    tooltip='Country:N'
).project(
    type= 'mercator',
    scale= 350,                          # Magnify
    center= [20,50],                     # [lon, lat]
    clipExtent= [[0, 0], [400, 300]],    # [[left, top], [right, bottom]]
).properties(
    title='Europe (Mercator)',
    width=400, height=300
)

background + main

alt.LayerChart(...)

In [48]:
import pandas as pd
import altair as alt
import panel as pn
import numpy as np
from vega_datasets import data
pn.extension('vega')

# -- Data loading --
data_files = {
    "Germany": "Country_csv/Germany.csv",
    "France": "Country_csv/France.csv",
    "Denmark": "Country_csv/Denmark.csv",
    "Belgium": "Country_csv/Belgium.csv"
}

# Load CSVs into a dictionary of dataframes
dataframes = {country: pd.read_csv(file) for country, file in data_files.items()}
dataframes_all = pd.concat([dataframes[country] for country in dataframes.keys()])

# -- Mapping of ISO 3166-1 alpha-3 codes to country names --
iso_to_country = {'DEU': 'Germany', 'FRA': 'France', 'DNK': 'Denmark', 'BEL': 'Belgium'}
country_to_iso = {v: k for k, v in iso_to_country.items()}

# Load map data
countries = alt.topo_feature(data.world_110m.url, 'countries')

# Dropdown for country and week
country_select = pn.widgets.Select(name='Select Country', options=list(dataframes.keys()), value='Germany')
week_slider = pn.widgets.Select(name='Select Week', options=[], value=None)

# Output panes
map_pane = pn.pane.Vega(sizing_mode='stretch_width', height=350)
bar_pane = pn.pane.Vega(sizing_mode='stretch_width', height=500)
df_pane = pn.pane.DataFrame(height=300)
df_song_detail = pn.pane.DataFrame(height=300)

# -- Reactive function --
def update_visuals(country):
    df = dataframes[country]
    weeks = sorted(df['week'].unique())
    week_slider.options = weeks
    week_slider.value = weeks[0] if weeks else None
    return

country_select.param.watch(lambda e: update_visuals(e.new), 'value')
update_visuals(country_select.value)

def create_map(selected_song, selected_week):
    if not selected_song or not selected_week:
        return alt.Chart().mark_text(text="Select a song to view map", align="center", dy=150
                                   ).properties(width=400, height=300)
    
    ids = pd.read_json('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/refs/heads/master/all/all.json')
    ids['geo'] = ids['alpha-3'].str.lower()

    #df_song = pd.DataFrame([['France', 1]], columns = ['name', 'streams'])
    # Filter data for selected song/week
    
    df_song = dataframes_all[(dataframes_all['track_name'] == selected_song) & 
                           (dataframes_all['week'] == selected_week)]
    df_song['name'] = df_song['country']
    df_song = df_song.merge(ids[['name', 'country-code']], how='left', on='name')

    # Background map (grey countries)
    background = alt.Chart(countries).mark_geoshape(
        fill='lightgrey',
        stroke='white'
    ).project(
    type='mercator',
    center=[10, 50],     # Slightly west and lower center for better Europe framing
    scale=500,           # Zoom in a bit more
    translate=[200, 150]  # Center the map visually (half of width and height)
    ).properties(
        width=400,
        height=300
    )

    # Countries with data
    choropleth = alt.Chart(countries).mark_geoshape(
        stroke='white'
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(df_song, key='country-code', fields=['streams', 'name'])
    ).encode(
        color=alt.Color('streams:Q', 
                       scale=alt.Scale(scheme='blues'), 
                       title="Streams"),
        tooltip=['name:N', 'streams:Q']
    ).project(
    type='mercator',
    center=[10, 50],     # Slightly west and lower center for better Europe framing
    scale=500,           # Zoom in a bit more
    translate=[200, 150]  # Center the map visually (half of width and height)
    ).properties(
        width=400,
        height=300
    )

    return (background + choropleth).configure_view(stroke=None)
# -- Modify your existing code --

# Add a song selector widget (will be populated in update_all)
song_select = pn.widgets.Select(name='Select Song', options=[], sizing_mode='stretch_width')

# -- Update the update_all function --
def update_all(event=None):
    country = country_select.value
    week = week_slider.value
    if not week:
        return

    df = dataframes[country]
    df_week = df[(df['week'] == week)]
    df_top_10 = df_week.head(10)
    
    # Update song selector options
    song_options = df_top_10['track_name'].tolist()
    song_select.options = song_options
    song_select.value = song_options[0] if song_options else None
    
    # Update bar chart (without selection)
    bars = alt.Chart(df_top_10).mark_bar().encode(
        x=alt.X('streams:Q', title='Number of Streams'),
        y=alt.Y('track_name:N', sort='-x', title='Song Title'),
        color='track_name:N',
        tooltip=['rank', 'track_name', 'artist_names', 'streams']
    ).properties(width=700, height=500)
    
    bar_pane.object = bars
    df_pane.object = df_week[['rank', 'track_name', 'artist_names', 'streams']]
    
    # Function to update details when song changes
    def update_song_details(event=None):
        selected_song = song_select.value
        if selected_song:
            # Update song details
            df_selected = df_week[df_week['track_name'] == selected_song]
            df_song_detail.object = df_selected[["rank", "track_name", "artist_names", "streams"]]
            
            # Update map
            map_pane.object = create_map(selected_song, week)
        else:
            df_song_detail.object = pd.DataFrame()
            map_pane.object = alt.Chart().mark_text(text="No song selected").properties(width=400, height=300)
    
    # Watch for song selection changes
    song_select.param.watch(update_song_details, 'value')
    
    # Initial update
    update_song_details()

# -- Update your layout --
app = pn.Column(
    pn.Row(country_select, week_slider),
    pn.Column(map_pane),
    pn.Row(song_select),
    pn.Column(bar_pane),
    pn.pane.Markdown("## Scrollable Song Rankings"),
    df_pane,
    pn.pane.Markdown("## Selected Song Details"),
    df_song_detail
)

# Trigger updates
country_select.param.watch(update_all, 'value')
week_slider.param.watch(update_all, 'value')
update_all()

app.servable()

/var/folders/mz/0vgr90l10rbgxrlhprqw1s0c0000gn/T/ipykernel_24657/1718014117.py:6: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension('vega')


/var/folders/mz/0vgr90l10rbgxrlhprqw1s0c0000gn/T/ipykernel_24657/1718014117.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_song['name'] = df_song['country']


Column
    [0] Row
        [0] Select(options=['Germany', 'France', ...], value='Germany')
        [1] Select(options=[1, 2, 3, 4, 5, ...], value=1)
    [1] Column
        [0] Vega(LayerChart, height=350, selection=Selection, sizing_mode='stretch_width')
    [2] Row
        [0] Select(options=['APT.', 'That’s So True',...], sizing_mode='stretch_width', value='APT.')
    [3] Column
        [0] Vega(Chart, height=500, selection=Selection, sizing_mode='stretch_width')
    [4] Markdown(str)
    [5] DataFrame(DataFrame, height=300)
    [6] Markdown(str)
    [7] DataFrame(DataFrame, height=300)